# Résumer un document

Commencez par télécharger le document que vous souhaitez résumer.

In [4]:
# Download a file
import wget

file_path = "my_document.pdf"
doc_url = "https://beta.gouv.fr/content/docs/mooc/23-lancez-vous-avec-demarches-simplifiees.pdf"

wget.download(doc_url, out=file_path)

Puis instancier la connexion à l'API Albert.

In [5]:
# Request configuration
import requests
import os

base_url = os.getenv("API_URL") # e.g. "http://localhost:8000/v1/"
api_key = os.getenv("API_KEY") # e.g. "your api key"

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}  # skip headers if no api_key is setup in config.ini file

Enfin pour vous importer le document dans une collection de notre base vectorielle à l'aide du endpoint POST `/v1/files`.

Vous devez spécifier le modèle d'embeddings qui sera utilisé pour vectoriser votre document. Vous pouvez trouver la liste des modèles avec le endpoint `/v1/models`. Les modèles d'embeddings sont indiqués avec le type _feature-extraction_.

Le endpoint POST `/v1/files` doit retourner un status _success_. 

In [8]:
# Upload a file
collection = "leo"
model = "BAAI/bge-m3"
params = {"collection": collection, "model": model} 

files = {'files': (os.path.basename(file_path), open(file_path, 'rb'), "application/pdf")}
response = session.post(f"{base_url}/files", params=params , files=files)

response.json()

{'object': 'list',
 'data': [{'object': 'upload',
   'id': 'cefd07f0-de07-439a-9b20-d3aae6985a3b',
   'filename': 'my_document.pdf',
   'status': 'success'}]}

Vous pouvez observer les fichiers que vous avez importer dans une collection à l'aide du endpoint GET `/v1/files`.

In [ ]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")

response.json()

Chaque fichier importé est associé un _file_id_, stocker le, il sera nécessaire pour la suite.

In [ ]:
file_id = response.json()["data"][0]["id"]

Pour résumer un document vous devez utiliser le _tool_ qui permet de fournir au modèle un ou plusieurs document préalablement importés : _UseFile_. Vous trouverez la liste des _tools_ ainsi que leur paramètres avec le endpoint GET `/v1/tools`.

In [7]:
# Display tools parameters
response = session.get(f"{base_url}/tools")
for tool in response.json()["data"]:
    if tool["id"] == "UseFiles":
        print(tool["description"])

Fill your prompt with file contents. Your prompt must contain "{files}" placeholder.

    Args:
        collection (str): Collection name.
        file_ids (List[str]): List of file ids in the selected collection.


In [31]:
# OpenAI client configuration
from openai import OpenAI

client = OpenAI(base_url=base_url, api_key=api_key)

Nous pouvons maintenant interroger le modèle avec le endpoint POST `/v1/chat/completions`

In [ ]:
# Chat completions
message = "Résume le document suivant : {files}"
user = "leo" # for chat history
data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "user": user,
    "messages": [{"role": "user", "content": message}],
    "stream": False,
    "n": 1,
    "tools": [
        {
            "function": {
                "name": "UseFiles",
                "parameters": {
                    "collection": collection,
                    "file_ids": [file_id]
                },
            },
            "type": "function",
        }
    ],
}

response = session.post(f"{base_url}/chat/completions", json=data)
print(response.json()["choices"][0]["content"])

In [48]:
print(response.json()["choices"][0]["message"]["content"])

Le document présente le service "Demarches-simplifiées" créé par la DINSIC (Direction interministérielle du numérique et des systèmes d'information et de communication) en 2015. Ce service en ligne permet aux acteurs publics de créer des formulaires en ligne pour les démarches administratives, sans nécessiter d'installation technique. 

L'objectif est de dématérialiser 100% des démarches administratives d'ici 2022. Pour y parvenir, Demarches-simplifiées permet aux acteurs publics de créer des formulaires en ligne de manière simple, en utilisant les ressources et données de la DINSIC. Les agents publics peuvent ainsi gérer les démarches en ligne sans avoir à posséder des compétences techniques en informatique.

Le service a déjà été adopté par plus de 500 organismes publics et plus de 5000 agents publics sont inscrits comme instructeurs. L'objectif est de dépasser les 10 000 démarches en ligne d'ici la fin de l'année 2018.

Le service est considéré comme une "Startup d'État" créée pour 

A l'aide de l'historique de chat disponible en récupérant l'ID retourné puisque j'ai initié une nouvelle conversation, je peux continuer a affiner sa réponse.

In [49]:
chat_id = response.json()["id"]

In [51]:
message = "Peux-tu me sortir les chiffres clefs du résumé que tu as fait ?"

data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "user": user,
    "id": chat_id,
    "messages": [{"role": "user", "content": message}],
    "stream": False,
    "n": 1,
}

response = session.post(f"{base_url}/chat/completions", json=data)
print(response.json()["choices"][0]["message"]["content"])

Oui, bien sûr !

Voici les chiffres clés extrait du résumé :

* 500 : Nombre d'organismes publics qui ont adopté le service
* 5000 : Nombre d'agents publics inscrits comme instructeurs
* 10 000 : Objectif de démarches en ligne à atteindre d'ici la fin de l'année 2018
* 2022 : Objectif de dématérialiser 100% des démarches administratives
